# Preparation 

In [ ]:
!pip install drain3

In [ ]:
import re
import pandas as pd

#from drain3 import TemplateMiner
import re
#from drain3.template_miner_config import TemplateMinerConfig
from multiprocessing import Pool
from tqdm import tqdm_notebook
from sklearn.metrics import precision_recall_fscore_support

#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
openstack1 = pd.read_csv('/content/drive/MyDrive/openstack/openstack1.csv')
openstack2 = pd.read_csv('/content/drive/MyDrive/openstack/openstack2.csv')
openstack_abnormal = pd.read_csv('/content/drive/MyDrive/openstack/abnormal.csv')

In [ ]:
openstack1.rename(columns={'Label':'Id'},inplace=True)
openstack2.rename(columns={'Label':'Id'},inplace=True)
openstack_abnormal.rename(columns={'Label':'Id'},inplace=True)

In [ ]:
m = []
m.append(openstack1)
m.append(openstack2)
m.append(openstack_abnormal)

for df in m:
    df['sub_content'] = df['Content'].str.replace(r'\[?instance:? [a-zA-Z0-9\-]+\]?','instance_id')
    df['sub_content'] = df['sub_content'].str.replace(r"\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}[^0-9]", 'IP')
    df['x'] = 0

In [ ]:
tc = TemplateMinerConfig()
tc.load('./drive/MyDrive/hdfs/drain3_BGL.ini') # this is config file, use the same setting for both BGL and Thunderbird 
tc.drain_extra_delimiters = ['\"']
tc.drain_depth = 6
tm=TemplateMiner(config=tc)

for df in m:
    df['Template_id'] = df['sub_content'].apply(lambda x: tm.add_log_message(x)['cluster_id'] )
    df['x'] = 1
for df in m:
    df['Template_id'] = df['sub_content'].apply(lambda x: tm.add_log_message(x)['cluster_id'] )
    df['x'] = 2

templates = [' '.join(tm.drain.id_to_cluster[i+1].log_template_tokens) for i in range(len(tm.drain.id_to_cluster))]

for df in m:
    df['Template_string'] = df['Template_id'].apply(lambda x: templates[x-1])



In [ ]:

for df in m:
    df['Time'] = pd.to_datetime(df['Time'], format = '%H:%M:%S.%f')
    df['diff'] = df['Time'].sub(df['Time'][0]).dt.total_seconds().mul(1000).astype(int)

In [ ]:
#create txt files
def create_file(dataset,save_PATH,f2 = False):
    dataset.sort_values('Time',inplace = True)
    f1 = dataset.groupby('Id')['Template_id'].agg(lambda x: ' '.join([str(i) for i in x])+'\n')
    if f2:
        f2 = dataset.groupby('Id')['diff'].agg(list).apply(lambda x: [i-min(x) for i in x])
        f2 = f2.apply((lambda x: ' '.join([str(i) for i in x])+'\n'))
        dataset = f1+'@@'+f2
    else:
        dataset = f1
    dataset = dataset.to_list()
    with open(save_PATH, 'w') as data:
        for i in dataset:
            data.write(i)

In [ ]:
create_file(openstack1,'/content/drive/MyDrive/openstack/openstack_train.txt')
create_file(openstack2,'/content/drive/MyDrive/openstack/openstack_test_normal.txt')
create_file(openstack_abnormal,'/content/drive/MyDrive/openstack/openstack_test_abnormal.txt')

In [ ]:
def countlen(path):
    m = 0
    with open(path) as data:
        for i in data.readlines():
            m+=1
    print(m)

In [ ]:
a = set([])

In [ ]:
for i in openstack1.groupby('Id')['Template_id'].agg(list).to_list():
    a.update(i)

for i in openstack2.groupby('Id')['Template_id'].agg(list).to_list():
    a.update(i)

for i in openstack_abnormal.groupby('Id')['Template_id'].agg(list).to_list():
    a.update(i)

In [ ]:
if len(a)-int(max(a))==0:
    print(max(a))
# there 22 diffferent tokens

22
